# fasthtml

> The fastest way to create an HTML app

`fasthtml` is a library for writing fast and scalable Starlette-powered web applications, without having to learn much (if any!) Starlette. Instead, you just use plain python functions for each page in your app -- you don't even need to learn Javascript. The finished app will be about as fast as a Python web server can be (which is pretty fast -- e.g. Instagram runs on Python), and you can create pretty much anything. This isn't one of those stripped-down dashboard making thingies.

This is a way to write real web applications, without the fuss.

To learn how to use it, please visit the [documentation](https://answerdotai.github.io/fasthtml/).

## Install

```sh
pip install python-fasthtml
```

## How to use

Import `fasthtml`, and you'll probably want the widgets from `fastcore.xml` too.

In [1]:
from fasthtml import *
from fastcore.xml import *

Create your app.

In [2]:
app = FastHTML()

Create your routes. The syntax is largely the same as the wonderful [FastAPI](https://fastapi.tiangolo.com/) (which is what you should be using instead of this if you're creating a JSON service. FastHTML is for mainly for making HTML web apps, not APIs).

Note that you need to include the types of your parameters, so that `FastHTML` knows what to pass to your function. Here, we're just expecting a string:

In [3]:
@app.get('/user/{nm}')
def get_nm(nm:str): return f"Good day to you, {nm}!"

Normally you'd save this into a file such as main.py, and then run it in `uvicorn` using:

```
uvicorn main:app
```

However, for testing, we can use Starlette's `TestClient` to try it out:

In [4]:
from starlette.testclient import TestClient

In [5]:
client = TestClient(app)
r = client.get('/user/Jeremy')
r

<Response [200 OK]>

TestClient uses `httpx` behind the scenes, so it returns a `httpx.Response`, which has a `text` attribute with our response body:

In [6]:
r.text

'Good day to you, Jeremy!'

FastHTML has special handling of tags created using `fastcore.xml`, so you can return web pages without worrying about Jinja, templates, or any of that stuff. This also means you can `pip install` styled rich component libraries, since it's all just pure python:

In [7]:
@app.get('/html/{idx}')
async def get_html(idx:int):
    return Body(
        H4("Wow look here"),
        P(f'It looks like you are visitor {idx}! Next is {idx+1}.')
    )

In [8]:
from IPython import display

In [9]:
display.HTML(client.get('/html/1').text)

## Features

Here's a brief demo of all the features of the library:

In [124]:
from starlette.responses import Response
from datetime import datetime
from fastcore.utils import *
from dataclasses import dataclass, asdict

In [125]:
def todict(req): return {k:str(v) for k,v in req.items()}

In [126]:
app = FastHTML()

@app.get("/")
def root(req): return todict(req.scope)

@app.get('/user/{nm}')
def get_nm(nm:str): return f"Good day to you, {nm}!"

In [127]:
client = TestClient(app)
r = client.get('/')
print(r.text)

{"type":"http","http_version":"1.1","method":"GET","path":"/","raw_path":"b'/'","root_path":"","scheme":"http","query_string":"b''","headers":"[(b'host', b'testserver'), (b'accept', b'*/*'), (b'accept-encoding', b'gzip, deflate, br'), (b'connection', b'keep-alive'), (b'user-agent', b'testclient')]","client":"['testclient', 50000]","server":"['testserver', 80]","extensions":"{'http.response.debug': {}}","state":"{}","app":"<starlette.applications.Starlette object at 0x12f39c990>","starlette.exception_handlers":"({<class 'starlette.exceptions.HTTPException'>: <bound method ExceptionMiddleware.http_exception of <starlette.middleware.exceptions.ExceptionMiddleware object at 0x12f39dcd0>>, <class 'starlette.exceptions.WebSocketException'>: <bound method ExceptionMiddleware.websocket_exception of <starlette.middleware.exceptions.ExceptionMiddleware object at 0x12f39dcd0>>}, {})","router":"<starlette.routing.Router object at 0x12f39c510>","endpoint":"<function _wrap_ep.<locals>._f at 0x129bef

In [128]:
client.get('/user/jph').text

'Good day to you, jph!'

In [129]:
@app.get('/html/{idx}')
async def get_html(idx:int):
    return Body(
        H4("Wow look here"),
        P(f'It looks like you are visitor {idx}! Next is {idx+1}.')
    )

In [130]:
display.HTML(client.get('/html/1').text)

In [131]:
reg_re_param("imgext", "ico|gif|jpg|jpeg|webm")

@app.get(r'/static/{path:path}{fn}.{ext:imgext}')
def get_img(fn:str, path:str, ext:str): return f"Getting {fn}.{ext} from /{path}"

In [132]:
client.get('/static/foo/jph.ico').text

'Getting jph.ico from /foo/'

In [133]:
ModelName = str_enum('ModelName', "alexnet", "resnet", "lenet")

app = FastHTML()
@app.get("/models/{nm}")
def model(nm:ModelName): return nm

@app.get("/files/{path}")
async def txt(path: Path): return path.with_suffix('.txt')

In [134]:
print(TestClient(app).get('/models/alexnet').text)

alexnet


In [135]:
print(TestClient(app).get('/files/foo').text)

foo.txt


In [136]:
fake_db = [{"name": "Foo"}, {"name": "Bar"}]

@app.get("/items/")
def read_item(idx:int|None = 0): return fake_db[idx]

In [137]:
print(TestClient(app).get('/items/?idx=1').text)

{"name":"Bar"}


In [138]:
print(TestClient(app).get('/items/').text)

{"name":"Foo"}


In [139]:
@app.get("/booly/")
def booly(coming:bool=True): return 'Coming' if coming else 'Not coming'

In [140]:
cli = TestClient(app)
print(cli.get('/booly/?coming=true').text)

Coming


In [141]:
print(cli.get('/booly/?coming=no').text)

Not coming


In [142]:
@app.get("/datie/")
def datie(d:date): return d

In [143]:
cli = TestClient(app)
date_str = "17th of May, 2024, 2p"
print(cli.get(f'/datie/?d={date_str}').text)

2024-05-17 14:00:00


In [144]:
@dataclass
class Bodie:
    a:int;b:str

In [145]:
@app.post("/bodie/{nm}/")
async def bodie(nm:str, data:Bodie):
    res = asdict(data)
    res['nm'] = nm
    return res

In [146]:
cli.post('/bodie/me', data=dict(a=1, b='foo')).text

'{"a":1,"b":"foo","nm":"me"}'

In [147]:
@app.get("/setcookie")
async def setc(req):
    now = datetime.now()
    res = Response(f'Set to {now}')
    res.set_cookie('now', str(now))
    return res

In [148]:
cli.get('/setcookie').text

'Set to 2024-05-19 01:33:57.762263'

In [149]:
@app.get("/getcookie")
async def getc(now:date): return f'Cookie was set at time {now.time()}'

In [150]:
cli.get('/getcookie').text

'Cookie was set at time 01:33:57.762263'

In [151]:
@app.get("/ua")
async def ua(user_agent:str): return user_agent

In [152]:
cli.get('/ua', headers={'User-Agent':'FastHTML'}).text

'FastHTML'